Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

**IMPORTANT: DO NOT COPY OR SPLIT CELLS.** If you do, you'll mess the autograder. If need more cells to work or test things out, create a new cell. You may add as many new cells as you need.

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and group below:

In [1]:
COURSE = "StatModels_2020_q3"
GROUP = "" # Either D2A or D2B
NAME = "" # Match your GitHub Classroom ID

---

#### Gonzalo G. Peraza Mues, 2020

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Probability

What is probability? There exist two current interpretations of probability: the frequentist interpretation and the bayesian interpretation. You are likely to be more familiar with the frequentist interpretation. Given a $n$ trial of an experiment, with $m$ different outcomes, the probability of outcome $i$ is
$$
P(i) \lim_{n\to\infty} \frac{n_i}{n},
$$
where $n_i$ is the number of occurrences of outcome $i$, subject to $\sum_i^m n_i = n$.

The problem is we often associate probabilities to events for which the frequentist interpretation makes little sense, as, for example, the probability of raining tomorrow. There is only a single occurrence for this event, either it rains, or it doesn't. In such cases, the bayesian interpretation makes more sense. In this interpretation, a probability is a quantified *degree of belief* of how likely is an event to occur. This belief must be based, either implicitly or explicitly, on certain assumptions (also called the *priori*) or evidence. In the rain example, the evidence is our past experience regarding rain around the same time of the year.

The frequentist interpretation is the [original one](https://en.wikipedia.org/wiki/Classical_definition_of_probability), in words of Laplace himself (1984):

>*Probability theory is nothing but common sense reduced to calculation. ... [Probability] is thus simply a fraction whose numerator is the number of favorable cases and whose denominator is the number of all the cases possible ... when nothing leads us to expect that any one of these cases should occur more than any other.*

<center><img src=https://upload.wikimedia.org/wikipedia/commons/thumb/3/30/AduC_197_Laplace_%28P.S.%2C_marquis_de%2C_1749-1827%29.JPG/180px-AduC_197_Laplace_%28P.S.%2C_marquis_de%2C_1749-1827%29.JPG></center>
<center><a href="https://en.wikipedia.org/wiki/Pierre-Simon_Laplace">Pierre-Simon Laplace</a><br>1814</center>


Let's remember some definitions from your Probability and Statistics course:

- **[Trial or experiment](https://en.wikipedia.org/wiki/Experiment_(probability_theory%29):**
  A single occurrence with an outcome that is uncertain until we observe it. 
  <br>*For example, rolling a single die.*
- **[Outcome](https://en.wikipedia.org/wiki/Outcome_(probability%29):**
  A possible result of a trial; one particular state of the world. What Laplace calls a **case.**
  <br>*For example:* `4`.
- **[Sample Space](https://en.wikipedia.org/wiki/Sample_space):**
  The set of all possible outcomes for the trial, typically denoted $\Omega$. 
  <br>*For example,* `{1, 2, 3, 4, 5, 6}`.
- **[Event](https://en.wikipedia.org/wiki/Event_(probability_theory%29):**
  A subset of outcomes that together have some property we are interested in.
  <br>*For example, the event "even die roll" is the set of outcomes* `{2, 4, 6}`. 
- **[Probability](https://en.wikipedia.org/wiki/Probability_theory):**
  As Laplace said, the probability of an event with respect to a sample space is the "number of favorable cases" (outcomes from the sample space that are in the event) divided by the "number of all the cases" in the sample space (assuming "nothing leads us to expect that any one of these cases should occur more than any other"). Since this is a proper fraction, probability will always be a number between 0 (representing an impossible event) and 1 (representing a certain event).
<br>*For example, the probability of an even die roll is 3/6 = 1/2.*

The basic rules for dealing with probabilities are:
1. $0\leq P(A) \leq 1$
2. $P(\Omega) = 1$
3. $P(\varnothing) = 0$
4. $P(A) + P(\bar{A}) = 1$
5. $P(A\cup B) = P(A) + P(B) - P(A\cap B)$

As an exercise, try proving that
- $P(A\cap B) \leq P(A)$
- $P(A\cup B) = P(A) + P(B)$ for disjoint sets
- If $A_0, A_1, \ldots, A_m$ are mutually exclusive and joint exhaustive (they define a partition of $\Omega$), then $P(A_0) = 1- \sum_{i=1}^m P(A_i)$
- Prove rule 2 using 4 and 5, or 2 and 5 to prove 4. This means rules above are not independent.

The *de facto* example of a probabilistic event, that can be approach through the frequentist interpretation, is the toss of a coin. Let's simulate a coin toss in Python using random numbers. The function `numpy.random.random` returns a random float in the interval $[0,1)$. We will discuss how does it does this later on. For now, just accept that it samples uniformly in such interval, meaning any float has equal probability of appearing.

In [5]:
# Test the rand function, repeatingly excecunting the cell should yield a different random value.
np.random.rand()

0.5076584822841247

In [8]:
def toss_coin():
    r = np.random.rand()
    if r < 0.5:
        return 'H'
    else:
        return 'T'

In [13]:
# Toss a coin!
toss_coin()

'H'

But we cannot obtain probabilities from a single toss coin, so let's modify the function to obtain a sequence of trials.

In [15]:
def toss_coin(n_trials):
    r = np.random.rand(n_trials)
    trials = np.zeros(r.shape,  dtype='unicode_')
    trials[r < 0.5] = 'H'
    trials[r >= 0.5] = 'T'
    return trials

In [17]:
n = 10
tosses = toss_coin(n)
print(tosses)

['H' 'H' 'T' 'H' 'H' 'T' 'T' 'H' 'H' 'T']


Now, the **empirical probability** for heads is

In [18]:
(tosses == 'H').sum() / tosses.size

0.6

Which is not 0.5. We'll come back to that in a bit, but before let's discuss theoretical probabilities a bit further. 

For a fair coin, the probability of landing heads (`H`) or tails (`T`) is 0.5. Thats the value used in the function. The probability 1/2 is the **theoretical** probability, that we need to distinguish from the actual **empirical** probability that can be calculated from a set of coin tosses. To find theoretical probabilities, we can use [sets](https://docs.python.org/3/library/stdtypes.html#set-types-set-frozenset) to specify the set of possible outcomes and the set of favorable outcomes of an experiment. The code below implements Laplace's quote directly: Probability is thus simply a fraction whose numerator is the number of favorable cases and whose denominator is the number of all the cases possible.

In [19]:
from fractions import Fraction

def P(event, space): 
    "The probability of an event, given a sample space."
    return Fraction(cases(favorable(event, space)), 
                    cases(space))

favorable = set.intersection # Outcomes that are in the event and in the sample space
cases     = len              # The number of cases is the length, or size, of a set

What's the probability of rolling an even number with a single six-sided fair die? Mathematicians traditionally use a single capital letter to denote a sample space; I'll use D for the die:

In [22]:
D     = {1, 2, 3, 4, 5, 6} # a sample space
even  = {   2,    4,    6} # an event

P(even, D)

Fraction(1, 2)

Good to confirm what we already knew. We can explore some other events:

In [23]:
prime = {2, 3, 5, 7, 11, 13}
odd   = {1, 3, 5, 7, 9, 11, 13}

In [24]:
P(odd, D)

Fraction(1, 2)

In [25]:
P((even | prime), D) # The probability of an even or prime die roll

Fraction(5, 6)

What about a deck of cards? Consider dealing a hand of five playing cards. An individual card has a rank and  suit, like `'J♥'` for the Jack of Hearts, and a `deck` has 52 cards:

In [26]:
suits = u'♥♠♦♣'
ranks = u'KQJT98765432A'
deck  = [r + s for r in ranks for s in suits]
len(deck)

52

In [ ]:
print(deck)

Now I want to define `Hands` as the sample space of all 5-card combinations from `deck`. The function `itertools.combinations` does most of the work; we than concatenate each combination into a space-separated string:

In [27]:
import itertools

def combos(items, n):
    "All combinations of n items; each combo as a space-separated str."
    return set(map(' '.join, itertools.combinations(items, n)))

Hands = combos(deck, 5)
len(Hands)

2598960

There are too many hands to look at them all, but we can sample:

In [28]:
import random
random.sample(Hands, 7)

C:\Users\gabri\AppData\Local\Temp\ipykernel_9108\1855166777.py:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(Hands, 7)


['K♥ J♦ T♦ 7♦ 5♠',
 'K♣ J♦ 5♥ 3♥ 2♦',
 'K♥ K♠ 8♣ 7♠ 5♦',
 'T♥ T♣ 5♠ 5♣ 4♠',
 'Q♦ Q♣ J♦ 8♠ A♠',
 'T♥ T♣ 5♠ 4♦ 2♠',
 'K♥ T♥ 6♠ 6♣ 4♣']

Now we can answer questions like the probability of being dealt a flush (5 cards of the same suit):

In [29]:
flush = {hand for hand in Hands if any(hand.count(suit) == 5 for suit in suits)}

P(flush, Hands)

Fraction(33, 16660)

Or the probability of four of a kind:

In [30]:
four_kind = {hand for hand in Hands if any(hand.count(rank) == 4 for rank in ranks)}

P(four_kind, Hands)

Fraction(1, 4165)

In [55]:
def isHandSequence(hand):
    indexList = []
    cards = hand.split(" ")
    for card in cards:
        indexList.append(ranks.index(card[0]))
    indexList.sort()
    for i in range(1, len(indexList)):
        if indexList[i] - indexList[i-1] != 1:
            return False
    return True
rankTuples = list(itertools.combinations(ranks, 2))

## Exercise: Poker

One application of counting includes computing probabilities of
poker hands. A poker hand consists of 5 cards drawn from the deck.
The order in which we receive these 5 cards is irrelevant. The number
of possible hands is thus
$$
{52 \choose 5} = 2598960
$$
Which is just what we found before.

We have just found the probability of a flush and of a four of a kind. Let's compute the probability of all possible hands in order of increasing probability (the less likely first). The possible hands are:
1. Royal Flush: A, K, Q, J, 10 all in the same suit. (`royal_flush`)
2. Straight Flush: Five cards in a sequence, all in the same suit. (`straight_flush`)
3. Four of a Kind: Exactly what it sounds like. (`four_kind`)
4. Full House: 3 of a kind with a pair. (`full_house`)
5. Flush: Any 5 cards of the same suit, but not in sequence. (`flush`)
6. Straight: Any 5 cards in sequence, but not all in the same suit. (`straight`)
7. Three of a Kind: Exactly what it sounds like. (`three_kind`)
8. Two Pair: Two pairs of cards. (`two_pair`)
9. One Pair: One pair of cards. (`one_pair`)
10. High Card: Anything else. (`high_card`)

Store all **sets** into the variables in parenthesis. You can check your answers in the wikipedia page [Poker probability](https://en.wikipedia.org/wiki/Poker_probability).

In [58]:
# Royal Flush
royal_flush = {hand for hand in Hands if ((hand.count("K") == 1 and hand.count("Q") == 1 and hand.count("J") == 1 and hand.count("T") == 1 and hand.count("A") == 1) and any(hand.count(suit) == 5 for suit in suits))}

In [59]:
# Straight Flush
straight_flush = {hand for hand in Hands if (isHandSequence(hand) and any(hand.count(suit) == 5 for suit in suits))}

In [60]:
# Four kind
four_kind = {hand for hand in Hands if any(hand.count(rank) == 4 for rank in ranks)}

In [61]:
# Full house
full_house = {hand for hand in Hands if any((hand.count(rankTuple[0]) == 3 and hand.count(rankTuple[1]) == 2) or (hand.count(rankTuple[1]) == 3 and hand.count(rankTuple[0]) == 2) for rankTuple in rankTuples)}

In [62]:
# Flush
# -40 because of straight_flush and royal flush
flush = {hand for hand in Hands if any(hand.count(suit) == 5 for suit in suits)}


In [64]:
# Straight
straight = {hand for hand in Hands if (isHandSequence(hand))}

In [65]:
print(f'Royal flush:    size={len(royal_flush)},       p={P(royal_flush, Hands)}    = {float(P(royal_flush, Hands))}')
print(f'Straight flush: size={len(straight_flush)},      p={P(straight_flush, Hands)}    = {float(P(straight_flush, Hands))}')
print(f'Four kind:      size={len(four_kind)},     p={P(four_kind, Hands)}    = {float(P(four_kind, Hands))}')
print(f'Full house:     size={len(full_house)},    p={P(full_house, Hands)}      = {float(P(full_house, Hands))}')
print(f'Flush:          size={len(flush)},    p={P(flush, Hands)} = {float(P(flush, Hands))}')
print(f'Straight:       size={len(straight)},   p={P(straight, Hands)}      = {float(P(straight, Hands))}')
#print(f'Three kind:     size={len(three_kind)},   p={P(three_kind, Hands)}     = {float(P(three_kind, Hands))}')
#print(f'Two pair:       size={len(two_pair)},  p={P(two_pair, Hands)}    = {float(P(two_pair, Hands))}')
#print(f'One pair:       size={len(one_pair)}, p={P(one_pair, Hands)}     = {float(P(one_pair, Hands))}')
#print(f'High card:      size={len(high_card)}, p={P(high_card, Hands)}   = {float(P(high_card, Hands))}')

Royal flush:    size=4,       p=1/649740    = 1.5390771693292702e-06
Straight flush: size=36,      p=3/216580    = 1.3851694523963431e-05
Four kind:      size=624,     p=1/4165    = 0.00024009603841536616
Full house:     size=3744,    p=6/4165      = 0.0014405762304921968
Flush:          size=5148,    p=33/16660 = 0.0019807923169267707
Straight:       size=9216,   p=192/54145      = 0.0035460337981346383


# Non-Equiprobable Outcomes

So far, we have accepted Laplace's assumption that *nothing leads us to expect that any one of these cases should occur more than any other*.
In real life, we often get outcomes that are not equiprobable--for example, a loaded die favors one side over the others.  We will introduce three more vocabulary items:

* [Frequency](https://en.wikipedia.org/wiki/Frequency_%28statistics%29): a non-negative number describing how often an outcome occurs. Can be a count like 5, or a ratio like 1/6.

* [Distribution](http://mathworld.wolfram.com/StatisticalDistribution.html): A mapping from outcome to frequency of that outcome. We will allow sample spaces to be distributions. 

* [Probability Distribution](https://en.wikipedia.org/wiki/Probability_distribution): A probability distribution
is a distribution whose frequencies sum to 1. Also called the Probability Mass Function (PMF).

Let's recover our coin toss example, but modify our equation further to allow the possibility of an unfair coin.

In [ ]:
def toss_coin(n_trials, p=0.5):
    r = np.random.rand(n_trials)
    trials = np.zeros(r.shape,  dtype='unicode_')
    trials[r < p] = 'H'
    trials[r >= p] = 'T'
    return trials

In [ ]:
n = 100
tosses = toss_coin(n, 0.4)
print(tosses)

How can we obtain the distribution of $\left\{ H,T \right\}$? One possibility is to use `numpy.unique`, other to use the Counter class.

In [ ]:
np.unique(tosses, return_counts=True)

In [ ]:
from collections import Counter

Counter(tosses)

Which has the advantage that we can specify a PMF directly.

In [ ]:
Counter({'H':0.4, 'T':0.6})

Using counter, we can redefine our previous utility functions.

In [ ]:
def cases(outcomes): 
    "The total frequency of all the outcomes."
    return sum(Counter(outcomes).values())

def favorable(event, space):
    "A distribution of outcomes from the sample space that are in the event."
    space = Counter(space)
    return Counter({x: space[x] 
                    for x in space if x in event})

def Fraction(n, d): return n / d

For example, here's the probability of rolling an even number with a crooked die that is loaded to prefer 6:

In [ ]:
Crooked = Counter({1: 0.1, 2: 0.1, 3: 0.1, 4: 0.1, 5: 0.1, 6: 0.5})

P(even, Crooked)

As another example, an [article](http://people.kzoo.edu/barth/math105/moreboys.pdf) gives the following counts for two-child families in Denmark, where `GB` means a family where the first child is a girl and the second a boy (I'm aware that not all births can be classified as the binary "boy" or "girl," but the data was reported that way):

    GG: 121801    GB: 126840
    BG: 127123    BB: 135138

In [ ]:
DK = Counter(GG=121801, GB=126840,
             BG=127123, BB=135138)
DK

In [ ]:
first_girl  = {'GG', 'GB'}
P(first_girl, DK)

In [ ]:
second_girl = {'GG', 'BG'}
P(second_girl, DK)

This says that the probability of a girl is somewhere between 48% and 49%. The probability of a girl is very slightly higher for the second child. 

Given the first child, are you more likely to have a second child of the same sex?

In [ ]:
same = {'GG', 'BB'}
P(same, DK)

Yes, but only by about 0.3%.

What about empirical PMF? For example, for our coin tosses. We can write a function to estimate such probabilities.

In [ ]:
def PMF(trials):
    outcomes, freq = np.unique(trials, return_counts=True)
    prob = freq / len(trials)
    return dict(zip(outcomes, prob))

In [ ]:
coin_pmf = PMF(tosses)
coin_pmf

And the correct way to visualize a PMF is either with a bar chart or with a stem plot.

In [ ]:
coin_pmf

In [ ]:
plt.bar(range(len(coin_pmf)), coin_pmf.values(), tick_label=[*coin_pmf]);

In [ ]:
plt.stem(coin_pmf.values(), use_line_collection=True)
plt.xticks(range(len(coin_pmf)), [*coin_pmf]);

Though a stem plot doesn't look very nice with only two outcomes. We can encapsulate this into a function.

In [ ]:
def plot_PMF(pmf, ptype='bars', label=None):
    prob = pmf.values()
    labels = [*pmf]
    
    if ptype == 'bars':
        plt.bar(range(len(pmf)), prob, tick_label=labels, label=label);
    elif ptype == 'stem':
        plt.stem(prob, use_line_collection=True)
        plt.xticks(range(len(pmf)), labels, label=label);
    else:
        raise NotImplementedError
    plt.ylabel('probability')

In [ ]:
# The empirical probabilities only equal the theoretical probabilities in the limit n->infty.
# This is called the law of large numbers.
# Try different number of trials and explore the convergence.
plot_PMF(PMF(toss_coin(10)))

> #### Challenge questions
> - How would you determine of a coin is fair or unfair?
> - How would you turn an unfair coin to a fair coin? Hint: Consider the possibility of several tosses.

For the previous questions, the concept of independence is important. Remember that if two events are independent, then
$$
P(A\cap B) = P(A)P(B)
$$
So, for example, for a double coin toss.

In [ ]:
double_toss_pmf = Counter(HH=0.25, TT=0.25, HT=0.25, TH=0.25)
# Two events
first_heads = {'HH', 'HT'}
second_tails = {'HT', 'TT'}

# Test for independence
P(first_heads&second_tails, double_toss_pmf) == P(first_heads, double_toss_pmf)*P(second_tails, double_toss_pmf)

# Discrete Random Variables

PMF describe discrete random variables. Mathematically, the PMF is defined as
$$
p_X(x_i) = P(X = x_i)
$$
When obtaining the empirical PMF from a dataset, we are estimating the theoretical PMF. Strictly, random variables must take numerical values, so in the coin example above, we must modify our function to return 1 for H and 0 for T. With the above restriction, it now makes sense to calculate expectations and variance of distribution.
\begin{align}
E[X] =& \sum_{x\in X(\Omega)} x P(X=x)\\
Var(X) =& E[(X - E[X]^2)^2] = E[X^2] - E[X]^2\\
=& \sum_{x\in X(\Omega)} x^2 P(X=x) - \left(\sum_{x\in X(\Omega)} x P(X=x)\right)^2
\end{align}

> #### Practice exercise
> 1. Calculate the expectation and variance of a coin flip and a die roll
> 2. Prove the second equality in the variance equation

Expectation is a linear function:
\begin{align}
E[\sum c_i X_i + b] = \sum c_i E[X_i] + b
\end{align}

The variance holds the following properties:
$$
Var(X) \geq 0\\
Var(c X + b) = c^2 Var(X)
$$

Furthermore, for **independent** random variables:
$$
E[\prod X_i] = \prod E[X_i]\\
Var(\sum X_i) = \sum Var(X_i)
$$

> #### Practice exercise
> 1. Prove the properties above.

As an example, we will use the The National Survey of Family Growth data, following some  of Allen Downey examples from [Think Stats 2e](https://greenteapress.com/wp/think-stats-2e/).

Since 1973 the U.S. Centers for Disease Control and Prevention (CDC)
have conducted the National Survey of Family Growth (NSFG), which is
intended to gather "information on family life, marriage and divorce, pregnancy, infertility, use of contraception, and men's and women's health. The survey results are used . . . to plan health services and health education programs, and to do statistical studies of families, fertility, and health." The NSFG has been conducted seven times; each deployment is called a
cycle. We will use data from Cycle 6, which was conducted from January
2002 to March 2003.

Pregnancy data from Cycle 6 of the NSFG is in a file called 2002FemPreg.pkl; it is a pickle python binary, meaning it contains an exported Python object, in this case a Pandas data frame. Each line in the data frame is a record that contains data about one pregnancy. It contains, among many more, the following variables:
- `caseid` is the integer ID of the respondent
- `prglngth` is the integer duration of the pregnancy in weeks.
- `outcome` is an integer code for the outcome of the pregnancy. The code 1 indicates a live birth.
- `pregordr` is a pregnancy serial number; for example, the code for a respondent's first pregnancy is 1, for the second pregnancy is 2, and so on.
- `birthord` is a serial number for live births; the code for a respondent's first child is 1, and so on. For outcomes other than live birth, this field is blank.
- `birthwgt_lb` and `birthwgt_oz` contain the pounds and ounces parts of the birth weight of the baby.
- `agepreg` is the mother's age at the end of the pregnancy.
- `finalwgt` is the statistical weight associated with the respondent. It is a floating-point value that indicates the number of people in the U.S. population this respondent represents.

In [ ]:
columns= ['caseid', 'prglngth', 'outcome', 'pregordr', 'birthord', 'birthwgt_lb', 'birthwgt_oz', 'agepreg', 'finalwgt']
preg = pd.read_pickle('data/2002FemPreg.pkl.zip')
preg = preg[columns]
preg.head()

We start by selecting records for live births, then splitting into first borns and other.

In [ ]:
live = preg[preg.outcome == 1]
first = live[live.birthord == 1]
other = live[live.birthord != 1]

In [ ]:
Counter(live.birthwgt_lb.dropna())

In [ ]:
PMF(live.birthwgt_lb.dropna())

In [ ]:
plot_PMF(PMF(live.birthwgt_lb.dropna()))

Now, comparing pregnancy length between first borns and others:

In [ ]:
first_pmf = PMF(first.prglngth.dropna())
other_pmf = PMF(other.prglngth.dropna())

plt.stem([*first_pmf.keys()], first_pmf.values(), label='first', use_line_collection=True, basefmt=" ", linefmt='--r', markerfmt='or')
plt.stem([*other_pmf.keys()], other_pmf.values(), label='other', use_line_collection=True, basefmt=" ", linefmt='--b', markerfmt='ob')
plt.xlim(27, 50)
plt.legend();

In [ ]:
plt.bar([*first_pmf.keys()], first_pmf.values(), label='first', alpha=0.4)
plt.bar([*other_pmf.keys()], other_pmf.values(), label='other', alpha=0.4)
plt.xlim(27, 50)
plt.legend();

By plotting the PMF instead of the histogram, we can compare the two
distributions without being mislead by the difference in sample size. Based
on this figure, first babies seem to be less likely than others to arrive on time
(week 39) and more likely to be a late (weeks 41 and 42).

> #### Exercise
> Write functions called `pmf_mean` and `pmf_var` that take a PMF as an input and compute the mean and variance. 

In [ ]:
def pmf_mean(pmf):
    # YOUR CODE HERE
    raise NotImplementedError()
    
def pmf_var(pmf):
    # YOUR CODE HERE
    raise NotImplementedError()

# Note: It would be nice to define a PMF class that holds the plot, mean, and var functions.

In [ ]:
# Check your functions with a die toss
die_pmf = {1:1/6, 2:1/6, 3:1/6, 4:1/6, 5:1/6, 6:1/6}

print(pmf_mean(die_pmf))
print(pmf_var(die_pmf))

## Exercise: Biased distribution

This exercise is found in Think Stats 2e by Allen Downey:

In most foot races, everyone starts at the same time. If you are a fast runner, you usually pass a lot of people at the beginning of the race, but after a few miles everyone around you is going at the same speed. When I ran a long-distance (209 miles) relay race for the first time, I noticed an odd phenomenon: when I overtook another runner, I was usually much faster, and when another runner overtook me, he was usually much faster.

At first I thought that the distribution of speeds might be bimodal; that is, there were many slow runners and many fast runners, but few at my speed.

Then I realized that I was the victim of a bias similar to the effect of class size. The race was unusual in two ways: it used a staggered start, so teams started at different times; also, many teams included runners at different levels of ability.

As a result, runners were spread out along the course with little relationship between speed and location. When I joined the race, the runners near me were (pretty much) a random sample of the runners in the race.

So where does the bias come from? During my time on the course, the chance of overtaking a runner, or being overtaken, is proportional to the difference in our speeds. I am more likely to catch a slow runner, and more likely to be caught by a fast runner. But runners at the same speed are unlikely to see each other.

Write a function called ObservedPmf that takes a Pmf representing the actual distribution of runners' speeds, and the speed of a running observer, and returns a new PMF representing the distribution of runners' speeds as seen by the observer.

Compute the distribution of speeds you would observe if you ran a relay race at 7 mph with this group of runners.

In [ ]:
# Obtain speeds from the James Joyce Ramble 10K in Dedham MA
speeds = np.loadtxt('data/speeds.txt')

pmf = PMF(speeds)
plot_PMF(pmf, label='actual speeds')
plt.xlabel('Speed (mph)')
plt.ylabel('PMF')
plt.legend();

In [ ]:
# The idea is to take the original distribution 
# and modify the probabilities to make them proportional to the
# relative speeds. Remeber to normalize the new distribution.
def ObservedPmf(pmf, speed):
    """Returns a new Pmf representing speeds observed at a given speed.

    The chance of observing a runner is proportional to the difference
    in speed.

    Args:
        pmf: distribution of actual speeds
        speed: speed of the observing runner

    Returns:
        Pmf object
    """
    # Always use a new copy
    new = pmf.copy()
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return new

In [ ]:
# Plot the new distribution, it should be bimodal.
biased = ObservedPmf(pmf, 7)
plot_PMF(biased, label='observed speeds')
plt.xlabel('Speed (mph)')
plt.ylabel('PMF')
plt.legend();

In [ ]:
# Find the modes ( two largest probabilities)
modes = sorted(biased.items(), key=lambda item: item[1])[-2:]
print(modes)
print(f"Modes are at {modes[0][0]} and {modes[1][0]}. Should have obtaines 5.25 and 5.7.")

## References and textual sources
- Seeing Theory, at https://seeing-theory.brown.edu/
- Peter Norvig probability pytudes, at https://github.com/norvig/pytudes